In [ ]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
import os
import requests
import json

# URL to the kaartbladindex.json file
json_url = "https://service.pdok.nl/rws/ahn/atom/downloads/dtm_05m/kaartbladindex.json"

# Set the output folder
output_folder = "c://temp//"

# Download and load JSON data directly from the URL
response = requests.get(json_url)
response.raise_for_status()  # Raise an error if the request fails
data = response.json()

# Extract all URLs
urls = [feature['properties']['url'] for feature in data['features'] if 'url' in feature['properties']]
print(len(urls))

# there are 1373 urls in file 
dem_urls = urls[:2] #Change this to 1373 in order to get all the files

def download_file(url, dest_folder="dems"):
    os.makedirs(dest_folder, exist_ok=True)
    filename = os.path.basename(url)
    local_path = os.path.join(dest_folder, filename)

    if not os.path.exists(local_path):
        print(f"Downloading: {filename}")
        response = requests.get(url, stream=True)
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    else:
        print(f"Already downloaded: {filename}")

    return local_path

def compute_hillshade(elevation, azimuth=315, altitude=45):
    x, y = np.gradient(elevation)
    slope = np.pi / 2.0 - np.arctan(np.sqrt(x**2 + y**2))
    aspect = np.arctan2(-x, y)

    azimuth_rad = np.radians(azimuth)
    altitude_rad = np.radians(altitude)

    shaded = (
        np.sin(altitude_rad) * np.sin(slope) +
        np.cos(altitude_rad) * np.cos(slope) * np.cos(azimuth_rad - aspect)
    )

    hillshade = 255 * shaded
    return np.clip(hillshade, 0, 255).astype(np.uint8)

def process_dem(input_path, output_path):
    with rasterio.open(input_path) as src:
        elevation = src.read(1, resampling=Resampling.bilinear)

        hillshade = compute_hillshade(elevation)

        profile = src.profile
        profile.update(
            dtype=rasterio.uint8,
            count=1,
            compress='lzw',
            nodata=0
        )

        with rasterio.open(output_path, "w", **profile) as dst:
            dst.write(hillshade, 1)

        print(f"Hillshade saved: {output_path}")

def main():
    for url in dem_urls:
        local_dem = download_file(url)
        base_name = os.path.splitext(os.path.basename(local_dem))[0]
        output_file = os.path.join(output_folder, f"hillshade_{base_name}.tif")
        process_dem(local_dem, output_file)

        # ✅ Delete the DEM file after processing
        if os.path.exists(local_dem):
            os.remove(local_dem)
            print(f"Deleted: {local_dem}")

if __name__ == "__main__":
    main()

Downloading: M_01GN2.tif


C:\Users\mtmde\AppData\Local\Temp\ipykernel_39204\2861931695.py:42: RuntimeWarning: overflow encountered in square
  slope = np.pi / 2.0 - np.arctan(np.sqrt(x**2 + y**2))


Hillshade saved: c://temp//hillshade_M_01GN2.tif
Deleted: dems\M_01GN2.tif
Downloading: M_02DZ1.tif
Hillshade saved: c://temp//hillshade_M_02DZ1.tif
Deleted: dems\M_02DZ1.tif
